In [1]:
from data_utils import *

def prepare(x_train, x_val, x_test):
    return x_train[:, 1:], x_val[:, 1:], x_test[:, 1:]

raw_train_data, raw_test_data = get_data(data_dir)     # 1. 从数据集内读数据
x_train, y_train, x_val, y_val, x_test = data_preprocessing(raw_train_data, raw_test_data)  # 2. 数据预处理
x_train, x_val, x_test = prepare(x_train, x_val, x_test)

raw_train_data shape:  (4320, 24)
raw_test_data shape:  (4320, 9)
month_data[0] shape:  (18, 480)
x_train shape:  (4521, 163)
y_train shape:  (4521, 1)
x_val shape:  (1131, 163)
y_val shape:  (1131, 1)
x_test shape:  (240, 163)


In [2]:
# 前向传播的构建
import tensorflow as tf

input_size = x_train.shape[1]
layer1_size, layer2_size = 64, 16
output_size = 1

def get_weight_variable(shape, regularizer):
    weights = tf.get_variable('weights', shape, initializer=tf.truncated_normal_initializer(stddev=0.002))
    if regularizer != None:
        tf.add_to_collection('loss', regularizer(weights))
    return weights

def forward_propagation(x, regularizer):
    
    with tf.variable_scope('layer1'):
        weights = get_weight_variable([input_size, layer1_size], regularizer)
        biases = tf.get_variable('biases', [layer1_size], initializer=tf.constant_initializer(0.0))
    layer1 = tf.nn.relu(tf.matmul(tf.cast(x, tf.float32), weights) + biases)
    
    with tf.variable_scope('layer2'):
        weights = get_weight_variable([layer1_size, layer2_size], regularizer)
        biases = tf.get_variable('biases', [layer2_size], initializer=tf.constant_initializer(0.0))
    layer2 = tf.nn.relu(tf.matmul(tf.cast(layer1, tf.float32), weights) + biases)
    
    with tf.variable_scope('layer3'):
        weights = get_weight_variable([layer2_size, output_size], regularizer)
        biases = tf.get_variable('biases', [output_size], initializer=tf.constant_initializer(0.0))
    layer3 = tf.nn.relu(tf.matmul(tf.cast(layer2, tf.float32), weights) + biases)
    
    return layer3

D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

In [3]:
batch_size = 500
learning_rate_base = 0.0005
learning_rate_decay = 0.999
moving_average_decay = 0.99
reg_rate = 5e10
iterations = 3e3

model_path = 'model.ckpt'

def train(x_train, y_train, x_val, y_val):
    
    x = tf.placeholder(tf.float32, [None, input_size], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 1], name='y-input')
    
    regularizer = tf.contrib.layers.l2_regularizer(reg_rate)
    y_pred = forward_propagation(x, regularizer)
    
    print(y_pred.shape)
    
    global_step = tf.Variable(0, trainable=False)
    
    variable_average = tf.train.ExponentialMovingAverage(moving_average_decay, global_step)
    variable_average_op = variable_average.apply(tf.trainable_variables())
    
    loss = tf.reduce_mean(tf.square(y_ - y_pred))
    learning_rate = tf.train.exponential_decay(learning_rate_base, global_step,
                                    x_train.shape[0] / batch_size, learning_rate_decay)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variable_average_op]):
        train_op = tf.no_op(name='train')
        
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(int(iterations)):
            X_batch, y_batch = None, None
            idx = np.random.choice(x_train.shape[0], batch_size, replace=True)
            X_batch = x_train[idx]
            y_batch = y_train[idx]
            _, loss_value, step = sess.run([train_op, loss, global_step],
                                           feed_dict={x:X_batch, y_:y_batch})
            if i % 100 == 0:
                print('After %d iterations, loss on x_train is: %g' %(step, loss_value))
                
            if i % 100 == 0:
                val_dict = {x: x_val, y_: y_val}
                _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict=val_dict)
                print('After %d iterations, mse on x_val is: %g' % (i, loss_value))
        print('y_pred shape: ', y_pred.shape)
        result = sess.run(y_pred, feed_dict={x: x_test})
        print(result.shape)
#         print('y_pred: ',sess.run(y_pred,feed_dict={x: x_val}))

In [4]:
train(x_train, y_train, x_val, y_val)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

(?, 1)
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
After 1 iterations, loss on x_train is: 738.832
After 0 iterations, mse on x_val is: 696.117
After 102 iterations, loss on x_train is: 718.145
After 100 iterations, mse on x_val is: 612.886
After 203 iterations, loss on x_train is: 637.741
After 200 iterations, mse on x_val is: 545.054
After 304 iterations, loss on x_train is: 486.665
After 300 iterations, mse on x_val is: 485.953
After 405 iterations, loss on x_train is: 128.656
After 400 iterations, mse on x_val is: 126.7